# Math 164
## Line Search Methods
### Golden Section
Assumes $f(x)$ is unimodal: only one local minimizer on $[a,b]$

In [86]:
import numpy as np
import math

def GoldenSection(f,a,b, tol=0, N=100):
    # f is a function to minimize, [a,b] is the range to minimize over

    # Define range and find number of necessary iterations (if not specified)
    R = b-a
    p = (3-math.sqrt(5))/2
    if tol > 0:
        N = math.ceil(math.log(tol/R, 1-p))
    
    # Set initial conditions
    p0,q0 = (a,f(a)),(b,f(b))
    p1,q1 = a+p*R, b-p*R
    p1,q1 = (p1, f(p1)),(q1,f(q1))

    # Iterate and update search range
    for n in range(N):
        if p1[1] < q1[1]:
            q0,q1 = q1,p1
            p1 = p0[0]+p*(q0[0]-p0[0])
            p1 = (p1,f(p1))
        else:
            p0,p1 = p1,q1
            q1 = q0[0]-p*(q0[0]-p0[0])
            q1 = (q1,f(q1))
    return (p0[0],q0[0])


g = lambda x: (x-3.347)**2
GoldenSection(g,-5,5)


(3.347, 3.3470000000000004)

### Fibonacci Method
Same assumptions as before

In [87]:
# Useful functions:

def Fibonacci(n):
    n = n+1
    return ((1+math.sqrt(5))**n - (1-math.sqrt(5))**n) / (2**n * math.sqrt(5))

def invFibonacci(F):
    if F < 2: return 1
    return round(math.log(math.sqrt(5)*(F-0.5),(1+math.sqrt(5))/2))-1

def pFib(k,N):
    if k == 1: return 1 - Fibonacci(N)/Fibonacci(N+1)
    elif k == N: return 0.5-0.00001
    else: return 1 - Fibonacci(N-k+1)/Fibonacci(N-k+2)

# Main function:

def FibonacciMethod(f,a,b,tol=0,N=100):
    # Define range and find number of necessary iterations (if not specified)
    R = b-a
    if tol > 0:
        N = invFibonacci(R/tol) # +/- 1
        # R/tol < fibN+1
    
    # Set initial conditions
    p = pFib(1,N)
    p0,q0 = (a,f(a)),(b,f(b))
    p1,q1 = a+p*R, b-p*R
    p1,q1 = (p1, f(p1)),(q1,f(q1))

    # Iterate and update search range
    for k in range(2,N+1):
        p = pFib(k,N)
        if p1[1] < q1[1]:
            q0,q1 = q1,p1
            p1 = p0[0]+p*(q0[0]-p0[0])
            p1 = (p1,f(p1))
        else:
            p0,p1 = p1,q1
            q1 = q0[0]-p*(q0[0]-p0[0])
            q1 = (q1,f(q1))
    return (p0[0],q0[0])

g = lambda x: (x-3.235)**2
FibonacciMethod(g,-5,5)

(3.235, 3.2350000000000003)

### Bisection Method
Same assumptions but also requires the derivative to be known

In [99]:
def BisectionMethod(df,a,b,tol=0,N=100):
    # Define range and find necessary number of iterations (if not specified)
    R = b-a
    if tol>0:
        N = math.ceil(math.log(tol/R, 0.5))
    
    # Set initial conditions
    p,q = a,b

    # Iterate and update search range
    for k in range(N):
        x = (p+q)/2
        if df(x) < 0:
            p = x
        elif df(x) > 0:
            q = x
        else:
            return (x,x)
    return (p,q)

g = lambda x: (x-4.78)**2
dg = lambda x: 2*(x-4.78)
BisectionMethod(dg,-5,10)

(4.78, 4.78)